# Feature engineering

## Open-rate
open_rate = “What % of potential store-days were actually open this week?”

### 1. load both datasets and peek

In [40]:
import pandas as pd
weekly_sales= pd.read_csv("../data/processed/weekly_sales_v1.csv")
daily_sales_kaggle= pd.read_csv("../data/raw/train.csv")
weekly_sales.head()


/var/folders/jn/dc5313mj0qs72qr08qqw3w3r0000gn/T/ipykernel_38331/1107802266.py:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  daily_sales_kaggle= pd.read_csv("../data/raw/train.csv")


,Date,Sales,Promo,Year,Month,Week_num,trend,StateHoliday,SchoolHoliday,Digital_spend,CPM,Digital_impressions,CTR,Digital_clicks,LandingRate,Product_page_views,TV_spend,OOH_spend
0,2013-01-06,26129335,0.000000,2013,1,1,1,1,0.740103,71476.013153,6.108456,1.170116e+07,0.009486,111002.906731,0.496163,55075.497241,0.0,0.0
1,2013-01-13,49275222,0.831102,2013,1,2,2,0,0.083471,78186.218005,6.657182,1.174464e+07,0.012347,145011.371049,0.504316,73131.526628,0.0,0.0
2,2013-01-20,34377765,0.000000,2013,1,3,3,0,0.000000,81560.325505,5.848056,1.394657e+07,0.007442,103783.699864,0.411973,42756.078058,0.0,0.0
3,2013-01-27,46040169,0.830758,2013,1,4,4,0,0.000000,114601.069498,5.604729,2.044721e+07,0.009471,193648.184854,0.449691,87081.888730,0.0,0.0
4,2013-02-03,38466029,0.000000,2013,2,5,5,0,0.019199,70428.442099,5.811603,1.211859e+07,0.010723,129948.869713,0.465507,60492.137175,0.0,0.0


In [41]:
daily_sales_kaggle.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [42]:
print(f"Weekly sales columns: {weekly_sales.columns}")
print(weekly_sales.dtypes)
print(f"daily sales columns: {daily_sales_kaggle.columns}")
print(daily_sales_kaggle.dtypes)

Weekly sales columns: Index(['Date', 'Sales', 'Promo', 'Year', 'Month', 'Week_num', 'trend',
       'StateHoliday', 'SchoolHoliday', 'Digital_spend', 'CPM',
       'Digital_impressions', 'CTR', 'Digital_clicks', 'LandingRate',
       'Product_page_views', 'TV_spend', 'OOH_spend'],
      dtype='object')
Date                    object
Sales                    int64
Promo                  float64
Year                     int64
Month                    int64
Week_num                 int64
trend                    int64
StateHoliday             int64
SchoolHoliday          float64
Digital_spend          float64
CPM                    float64
Digital_impressions    float64
CTR                    float64
Digital_clicks         float64
LandingRate            float64
Product_page_views     float64
TV_spend               float64
OOH_spend              float64
dtype: object
daily sales columns: Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'Sch

In [43]:
# convert to datetime
weekly_sales["Date"]= pd.to_datetime(weekly_sales["Date"], errors="coerce")
weekly_sales["weekday"] = weekly_sales["Date"].dt.dayofweek
weekly_sales[["Date","weekday","Week_num"]]

,Date,weekday,Week_num
0,2013-01-06,6,1
1,2013-01-13,6,2
2,2013-01-20,6,3
3,2013-01-27,6,4
4,2013-02-03,6,5
...,...,...,...
130,2015-07-05,6,27
131,2015-07-12,6,28
132,2015-07-19,6,29
133,2015-07-26,6,30


Now we have to convert daily sales date to datetime and add week start column for later merge with weekly_sales

In [ ]:
# kaggle dataset datetime
daily_sales_kaggle.head()
daily_sales_kaggle["Date"]= pd.to_datetime(daily_sales_kaggle["Date"], errors="coerce")
daily_sales_kaggle["week_start"]= daily_sales_kaggle["Date"].dt.to_period("W-SUN")
daily_sales_kaggle["week_start"] = daily_sales_kaggle["week_start"].dt.to_timestamp()
daily_sales_kaggle["week_day_start"] = daily_sales_kaggle["week_start"].dt.dayofweek

daily_sales_kaggle.sort_values("Date")[["Store","Date","week_start","Open","week_day_start"]].head(10)

,Store,Date,week_start,Open,week_day_start
1017208,1115,2013-01-01,2012-12-31,0,0
1016473,379,2013-01-01,2012-12-31,0,0
1016472,378,2013-01-01,2012-12-31,0,0
1016471,377,2013-01-01,2012-12-31,0,0
1016470,376,2013-01-01,2012-12-31,0,0
1016469,375,2013-01-01,2012-12-31,0,0
1016468,374,2013-01-01,2012-12-31,0,0
1016467,373,2013-01-01,2012-12-31,0,0
1016466,372,2013-01-01,2012-12-31,0,0
1016465,371,2013-01-01,2012-12-31,0,0


In [52]:
# A1) Shift the weekly label to the Sunday that ends the week
daily_sales_kaggle["week_end"] = daily_sales_kaggle["week_start"] + pd.Timedelta(days=6)

# A2) Sanity check: Sunday == 6
daily_sales_kaggle["week_end_dow"] = daily_sales_kaggle["week_end"].dt.dayofweek
daily_sales_kaggle.sort_values("Date")[["Store","Date","week_start","week_end","week_end_dow"]].head(8)


,Store,Date,week_start,week_end,week_end_dow
1017208,1115,2013-01-01,2012-12-31,2013-01-06,6
1016473,379,2013-01-01,2012-12-31,2013-01-06,6
1016472,378,2013-01-01,2012-12-31,2013-01-06,6
1016471,377,2013-01-01,2012-12-31,2013-01-06,6
1016470,376,2013-01-01,2012-12-31,2013-01-06,6
1016469,375,2013-01-01,2012-12-31,2013-01-06,6
1016468,374,2013-01-01,2012-12-31,2013-01-06,6
1016467,373,2013-01-01,2012-12-31,2013-01-06,6


In [53]:
# B1) Unique Sundays in each
wk_sundays = set(pd.to_datetime(weekly_sales["Date"]).unique())
dl_sundays = set(daily_sales_kaggle["week_end"].unique())

# B2) Intersection size
len_wk = len(wk_sundays)
len_dl = len(dl_sundays)
len_inter = len(wk_sundays & dl_sundays)
(len_wk, len_dl, len_inter)


(135, 135, 135)